In [26]:
import sql_py_defs as sql
import matplotlib.pyplot as plt
import numpy as np
import keyring

In [27]:
pw = keyring.get_password("mysql", "ellaguise")
connection = sql.create_db_connection("localhost", "root", pw, "agn_samp")

MySQL Database connection successful


In [28]:
q1 = """
SELECT z,lum FROM agn_samp_z_5_area_10;
"""

results = sql.read_query(connection, q1)
z = [results[i][0] for i in range(len(results))]
lum = [results[i][1] for i in range(len(results))]

In [29]:
log_sig_hat_sq_rf = [8. - 0.27*(lum[i]) + (0.47+np.log10(1+z[i])) for i in range(len(z))]
log_tau_rf = [-8.13 + 0.24*(lum[i]) + (0.34*np.log10(1+z[i])) for i in range(len(z))] 

tau = [np.power(10., log_tau_rf[i])*(1+z[i]) for i in range(len(z))]
sig_hat_sq = [np.power(10., log_sig_hat_sq_rf[i])/(1+z[i]) for i in range(len(z))]
sigma = [np.sqrt(sig_hat_sq[i]*tau[i]/2.) for i in range(len(z))]

In [31]:
# Create New Table for DRW information

create_agn_table = """
CREATE TABLE agn_DRW_z_5_area_10 (
  id INT PRIMARY KEY,
  tau FLOAT NOT NULL,
  sig_hat FLOAT NOT NULL,
  sigma FLOAT NOT NULL
  );
 """

#sql.execute_query(connection, create_agn_table)

sql_ins = '''
    INSERT INTO agn_DRW_z_5_area_10 (id, tau, sig_hat, sigma) 
    VALUES (%s, %s, %s, %s)
    '''

    
val = [(int(i), float(np.log10(tau[i])), 0.5*float(np.log10(sig_hat_sq[i])), float(np.log10(sigma[i]))) for i in range(len(tau))]


sql.execute_list_query(connection, sql_ins, val)

Query successful
Query successful
